In [89]:
import pandas as pd 
import numpy as np 
from catboost import CatBoostClassifier,Pool,cv,utils

# import sys
# sys.path.append('C:\\Users\peter\\Documents\\MyProjects\\PyProj\\Trav\\spel')
# import V75_scraping as vs

In [90]:
dforg = pd.read_csv('..//all_data.csv')     
print(dforg.columns)

Index(['datum', 'avd', 'bana', 'häst', 'kusk', 'streck', 'vodds', 'podds',
       'kr', 'spår', 'dist', 'lopp_dist', 'start', 'ålder', 'kön', 'plac',
       'pris', 'h1_dat', 'h1_kusk', 'h1_bana', 'h1_spår', 'h1_plac', 'h1_pris',
       'h1_odds', 'h1_kmtid', 'h2_dat', 'h2_kusk', 'h2_bana', 'h2_spår',
       'h2_plac', 'h2_pris', 'h2_odds', 'h2_kmtid', 'h3_dat', 'h3_kusk',
       'h3_bana', 'h3_spår', 'h3_plac', 'h3_pris', 'h3_odds', 'h3_kmtid',
       'h4_dat', 'h4_kusk', 'h4_bana', 'h4_spår', 'h4_plac', 'h4_pris',
       'h4_odds', 'h4_kmtid', 'h5_dat', 'h5_kusk', 'h5_bana', 'h5_spår',
       'h5_plac', 'h5_pris', 'h5_odds', 'h5_kmtid', 'h1_dist', 'h2_dist',
       'h3_dist', 'h4_dist', 'h5_dist', 'bins', 'h1_auto', 'h2_auto',
       'h3_auto', 'h4_auto', 'h5_auto', 'h1_perf', 'h2_perf', 'h3_perf',
       'h4_perf', 'h5_perf', 'senast', 'delta1', 'delta2', 'delta3', 'delta4',
       'startnr'],
      dtype='object')


In [91]:
### Features som inte används vid träning
def remove_features(df,remove_mer=[]):
    #remove_mer=['h5_perf','h5_auto','h4_perf','h4_auto', 'h3_perf', 'h2_perf']
    df.drop(['startnr','vodds','podds','bins','h1_dat','h2_dat','h3_dat','h4_dat','h5_dat'],axis=1,inplace=True) #
    if remove_mer:
        df.drop(remove_mer,axis=1,inplace=True)
    
    # df=check_unique(df.copy())
    # df=check_corr(df.copy())
    return df

In [92]:
 ## byt ut alla NaN till text för cat_features
def replace_NaN(X_train,X_test=None, cat_features=[]):
    # print('cat_features',cat_features)
    X_train[cat_features]=X_train[cat_features].fillna('missing')
    if X_test is not None:  ## om X_test är med
        X_test[cat_features]=X_test[cat_features].fillnal('missing')    ### byt ut None-värden till texten 'None

    return X_train,X_test

In [93]:
# create basic and clean X_train, X_test etc that can be used as is
def basic_data(df, frac=0.25):
    X_train,X_test,y_train,y_test = None,None,None,None
    dfnew = remove_features(df.copy())
    dfnew['plac'] = (dfnew.plac==1) * 1
    cat_features = list(dfnew.loc[:,df.dtypes=='O'].columns)
    dfnew,_ = replace_NaN(dfnew.copy(), cat_features=cat_features)    
    
    alla_datum = df.datum.unique()
    split_dat = alla_datum[int(len(alla_datum)* (1 - 0.25))]     # större än split_dat är test

    X_train = dfnew.loc[dfnew.datum <= split_dat].copy()
    y_train=X_train.plac
    X_train.drop('plac',axis=1,inplace=True)
    
    X_test = dfnew.loc[dfnew.datum > split_dat].copy()
    y_test=X_test.plac
    X_test.drop('plac',axis=1,inplace=True)
    
    return X_train,X_test, y_train,y_test

# split_dat = dforg.datum.unique()[int(len(dforg.datum.unique())* (1 - 0.25))]  
# dforg.loc[dforg.datum<'2021-09-18']
# X_train,X_test,y_train,y_test = basic_data(dforg.copy())

In [94]:
# Handle ekipage (häst and kusk)
# Set a smooth mean value to the features in X_train  ##
def calc_smooth_mean(X, y, by, m=100, tot_mean=None):
    Xcopy = X.copy()
    Xcopy['plac'] = y

    # Compute the number of values and the mean of each group
    agg = Xcopy.groupby(by)['plac'].agg(['count', 'mean'])
    counts = agg['count']
    means = agg['mean']

    # Compute the "smoothed" means
    smooth = (counts * means + m * tot_mean) / (counts + m)
    
    return smooth.to_dict()


from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer


In [95]:
# df skall innehålla datum,avd,vodds
def proba_order_score(df_, y,proba):
    kassa=200
    df = df_.copy()
    df['proba'] = proba[:,1]
    df['f'] = (df.proba*df.vodds - 1) / (df.vodds-1)  # kelly formel
    df['spela'] = df.f >0
    df['insats'] = df.spela * df.f * kassa

    df.sort_values(['datum','avd','proba'],ascending=[True,True,False],inplace=True)
    proba_order=df.groupby(['datum','avd']).proba.cumcount()

    df['prob_order']=proba_order+1
    df['y'] = y
    
    return df, df.loc[df.y==1].prob_order.mean()   # mean vann per avd
    


# Kolla olika features och dess betydelse på resultatet
## data

In [96]:

dforg = pd.read_csv('..\\all_data.csv')     
#
X_train,X_test,y_train,y_test = basic_data(dforg.copy())
cat_features = list(X_train.loc[:,X_train.dtypes=='O'].columns)


In [97]:
# datum, avd, ekipage (raw, kodat) - blir det skillnad med och utan 'streck' 
# Spara alla värden nedan                           proba_order         AUC                 Accuracy
# 1.  datun, avd, häst och kusk                     3.1316964285714284  0.8154899734619122  0.7407225221611429                     
# 2.  ej avd                                        3.146205357142857   0.8151640292105853  0.7406341992435652
# 3.  ej datum+avd   (bara avd borde ge sämre res)  3.1361607142857144  0.8162590757456829  0.7453960511909773 
# 4.  datum, raw_ekipage                            3.1573660714285716  0.8154180216042461  0.7378602742086335
# 5.  ej datum+avd  med raw_ekipage                 3.166294642857143   0.8143806767212379  0.737153650575579 
# 6.  datum, kodat_ekipage                          VÄNTA
# 7.  ej datum+avd, kodat_ekipage                   VÄNTA
# 8.  bästa val enligt ovan med kodad bana          VÄNTA
# 9.  gör en cv (timeSeries) med bästa val (val=3)  0.2426               0.812119            -----
     
cb = CatBoostClassifier(iterations=2000, early_stopping_rounds=100, use_best_model=True,auto_class_weights='Balanced', 
                        custom_metric=['Logloss','Accuracy','Recall','Precision','F1','AUC'], eval_metric='Accuracy', random_state=2021, verbose=False)

## De olika modellern med/utan vissa features

In [98]:
# 1.    datum, avd, häst och kusk
res=cb.fit(X_train,y_train,eval_set=(X_test,y_test),cat_features=cat_features)
logloss, auc, accuracy = res.get_best_score()['validation']['Logloss'],res.get_best_score()['validation']['AUC'],res.get_best_score()['validation']['Accuracy']

X_test[['datum','avd','vodds']] = dforg[['datum','avd','vodds']]
X,prob_score = proba_order_score(X_test,y_test,res.predict_proba(X_test))
print(prob_score, auc, accuracy, logloss)

Custom logger is already specified. Specify more than one logger at same time is not thread safe.

3.1316964285714284 0.8154899734619122 0.7407225221611429 0.5297606225786045


In [100]:
# 2.    drop avd
X_train,X_test,y_train,y_test = basic_data(dforg.copy())

X_train.drop(['avd'],axis=1,inplace=True)
X_test.drop(['avd'],axis=1,inplace=True)
cat_features = list(X_train.loc[:,X_train.dtypes=='O'].columns)
res=cb.fit(X_train,y_train,eval_set=(X_test,y_test),cat_features=cat_features)
logloss, auc, accuracy = res.get_best_score()['validation']['Logloss'],res.get_best_score()['validation']['AUC'],res.get_best_score()['validation']['Accuracy']
X_test[['datum','avd','vodds']] = dforg[['datum','avd','vodds']]
X,prob_score = proba_order_score(X_test,y_test,res.predict_proba(X_test))
print(prob_score, auc, accuracy, logloss)

Custom logger is already specified. Specify more than one logger at same time is not thread safe.

3.146205357142857 0.8151640292105853 0.7406341992435652 0.5303265275620271


In [104]:
# 3.  ej datum+avd
X_train,X_test,y_train,y_test = basic_data(dforg.copy())
X_train.drop(['datum','avd'],axis=1,inplace=True)
X_test.drop(['datum','avd'],axis=1,inplace=True)
cat_features = list(X_train.loc[:,X_train.dtypes=='O'].columns)

res=cb.fit(X_train,y_train,eval_set=(X_test,y_test),cat_features=cat_features)
logloss, auc, accuracy = res.get_best_score()['validation']['Logloss'],res.get_best_score()['validation']['AUC'],res.get_best_score()['validation']['Accuracy']
proba = res.predict_proba(X_test)
X_test[['datum','avd','vodds']] = dforg[['datum','avd','vodds']]
X,prob_score = proba_order_score(X_test,y_test, proba)
print(prob_score, auc, accuracy, logloss)

3.1361607142857144 0.8162590757456829 0.7453960511909773 0.5315257039171241


In [105]:
# 4.  datum + raw_ekipage
X_train,X_test,y_train,y_test = basic_data(dforg.copy())
X_train.drop(['avd'],axis=1,inplace=True)
X_test.drop(['avd'],axis=1,inplace=True)
X_train['ekipage'] = X_train['häst'].str.cat(X_train.kusk,sep=',')
X_train.drop(['häst','kusk'],axis=1,inplace=True) 
X_test['ekipage'] = X_test['häst'].str.cat(X_test.kusk,sep=',')
X_test.drop(['häst','kusk'],axis=1,inplace=True) 

cat_features = list(X_train.loc[:,X_train.dtypes=='O'].columns)

res=cb.fit(X_train,y_train,eval_set=(X_test,y_test),cat_features=cat_features)
logloss, auc, accuracy = res.get_best_score()['validation']['Logloss'],res.get_best_score()['validation']['AUC'],res.get_best_score()['validation']['Accuracy']
proba = res.predict_proba(X_test)
X_test[['datum','avd','vodds']] = dforg[['datum','avd','vodds']]
X,prob_score = proba_order_score(X_test,y_test,proba)
print(prob_score, auc, accuracy, logloss)

3.1573660714285716 0.8154180216042461 0.7378602742086335 0.5348079567997289


In [107]:
# 5.  ej datum+avd  med raw_ekipage
X_train,X_test,y_train,y_test = basic_data(dforg.copy())
X_train.drop(['datum','avd'],axis=1,inplace=True)
X_test.drop(['datum','avd'],axis=1,inplace=True)
X_train['ekipage'] = X_train['häst'].str.cat(X_train.kusk,sep=',')
X_train.drop(['häst','kusk'],axis=1,inplace=True) 
X_test['ekipage'] = X_test['häst'].str.cat(X_test.kusk,sep=',')
X_test.drop(['häst','kusk'],axis=1,inplace=True) 

cat_features = list(X_train.loc[:,X_train.dtypes=='O'].columns)

res=cb.fit(X_train,y_train,eval_set=(X_test,y_test),cat_features=cat_features)
logloss, auc, accuracy = res.get_best_score()['validation']['Logloss'],res.get_best_score()['validation']['AUC'],res.get_best_score()['validation']['Accuracy']
proba = res.predict_proba(X_test)
X_test[['datum','avd','vodds']] = dforg[['datum','avd','vodds']]
X,prob_score = proba_order_score(X_test,y_test, proba)
print(prob_score, auc, accuracy, logloss)

3.166294642857143 0.8143806767212379 0.737153650575579 0.5373711001782585


## cv
p9.  gör en cv med bästa val enl ovan  val=3 (ej datum+avd)

In [16]:
X_train,X_test,y_train,y_test = basic_data(dforg.copy())
X_train.drop(['datum','avd'],axis=1,inplace=True)
X_test.drop(['datum','avd'],axis=1,inplace=True)
cat_features = list(X_train.loc[:,X_train.dtypes=='O'].columns)

df = pd.concat([X_train,X_test],axis=0)
y = pd.concat([y_train,y_test])
cv_pool = Pool(df,y,cat_features=cat_features)

params = {
         'use_best_model': True,
         'eval_metric' : 'AUC',
         "loss_function": "Logloss",
         'early_stopping_rounds': 100,
         'verbose': 50,
}


In [17]:

cv_score =cv(pool=cv_pool, 
   params=params, 
   dtrain=None, 
   iterations=2000, 
   num_boost_round=None,
   fold_count=5, 
   nfold=None,
   inverted=False,
   partition_random_seed=0,
   seed=2021, 
   shuffle=False, 
   logging_level=None, 
   stratified=True,
   as_pandas=True,
   type='TimeSeries')

0:	test: 0.6855664	best: 0.6855664 (0)
50:	test: 0.8097489	best: 0.8097489 (50)
100:	test: 0.8106969	best: 0.8121190 (77)
150:	test: 0.8099697	best: 0.8121190 (77)	total: 56.9s	remaining: 11m 36s
Stopped by overfitting detector  (100 iterations wait)


In [18]:
cv_score

,iterations,test-AUC-mean,test-AUC-std,test-Logloss-mean,test-Logloss-std,train-Logloss-mean,train-Logloss-std
0,0,0.685566,0.031138,0.659116,0.001195,0.659082,0.001414
1,1,0.785142,0.009562,0.623803,0.001406,0.623434,0.001732
2,2,0.793489,0.010459,0.594773,0.001503,0.594257,0.002040
3,3,0.795764,0.008138,0.570290,0.001322,0.569590,0.000968
4,4,0.798462,0.008920,0.545006,0.000998,0.544244,0.000614
...,...,...,...,...,...,...,...
173,173,0.809656,0.005633,0.240312,0.002036,0.219811,0.009642
174,174,0.809653,0.005669,0.240320,0.002044,0.219681,0.009647
175,175,0.809649,0.005703,0.240319,0.002056,0.219620,0.009649
176,176,0.809561,0.005666,0.240335,0.002056,0.219528,0.009628


In [19]:
from IPython.display import display
print(dforg.datum.max())
display(cv_score[cv_score['test-Logloss-mean'].min() == cv_score['test-Logloss-mean']])
display(cv_score[cv_score['test-AUC-mean'].max() == cv_score['test-AUC-mean']])

2021-09-18


,iterations,test-AUC-mean,test-AUC-std,test-Logloss-mean,test-Logloss-std,train-Logloss-mean,train-Logloss-std
114,114,0.810767,0.006555,0.239625,0.00228,0.226369,0.006912


,iterations,test-AUC-mean,test-AUC-std,test-Logloss-mean,test-Logloss-std,train-Logloss-mean,train-Logloss-std
77,77,0.812119,0.006616,0.2426,0.001858,0.234775,0.004969


## FLAML

In [108]:
X_train,X_test,y_train,y_test = basic_data(dforg.copy())
X_train.drop(['avd'],axis=1,inplace=True)
X_test.drop(['avd'],axis=1,inplace=True)
cat_features = list(X_train.loc[:,X_train.dtypes=='O'].columns)

In [109]:
from flaml import AutoML
automl = AutoML()
automl.fit(X_train,y_train, X_val=X_test, y_val=y_test, task='classification',  metric='roc_auc', ensemble=True, 
            n_jobs=5,split_type='time',seed=2021,early_stop=True, time_budget=240,  max_iter=2000000)

[flaml.automl: 09-23 18:10:23] {1427} INFO - Evaluation method: holdout
[flaml.automl: 09-23 18:10:23] {1473} INFO - Minimizing error metric: 1-roc_auc
[flaml.automl: 09-23 18:10:23] {1505} INFO - List of ML learners in AutoML Run: ['lgbm', 'rf', 'catboost', 'xgboost', 'extra_tree', 'lrl1']
[flaml.automl: 09-23 18:10:23] {1735} INFO - iteration 0, current learner lgbm
[flaml.automl: 09-23 18:10:23] {1914} INFO -  at 1.2s,	best lgbm's error=0.1918,	best lgbm's error=0.1918
[flaml.automl: 09-23 18:10:23] {1735} INFO - iteration 1, current learner lgbm
[flaml.automl: 09-23 18:10:24] {1914} INFO -  at 1.7s,	best lgbm's error=0.1918,	best lgbm's error=0.1918
[flaml.automl: 09-23 18:10:24] {1735} INFO - iteration 2, current learner lgbm
[flaml.automl: 09-23 18:10:24] {1914} INFO -  at 2.2s,	best lgbm's error=0.1918,	best lgbm's error=0.1918
[flaml.automl: 09-23 18:10:24] {1735} INFO - iteration 3, current learner xgboost
[flaml.automl: 09-23 18:10:24] {1914} INFO -  at 2.5s,	best xgboost's e

In [110]:
# automl.best_loss, 1-automl.best_loss
X_test[['datum','avd','vodds']] = dforg[['datum','avd','vodds']]
proba = automl.predict_proba(X_test)
X,prob_score = proba_order_score(X_test,y_test,proba)
prob_score,1-automl.best_loss

(3.125, 0.818371201502579)

In [111]:
prob_score, 1-automl.best_loss

(3.125, 0.818371201502579)

0.1812145496935037, 0.8187854503064963   utan datum o avd
0.18135056907378333, 0.8186494309262167  med datum utan avd
0.27235127223966504, 0.727648727760335   utan streck, datum, avd
3.7142857142857144, 0.26984749943933606  utan streck med datum